## `map_countrycodes.py`
1. Read the csv file from the `Output/output02-gnoc_networks-colsdropped.csv` into pandas dataframe
2. Add new column CountryCode in dataframe by performing jmespath search in `CityDatabase/regions.json` 
3. Write file to `Output/output03-gnoc_networks-countrycodes.csv`

In [1]:
import pandas as pd
import json
import jmespath
import numpy as np
import os.path
import time

In [2]:
df = pd.read_csv("Output/output02-gnoc_networks-colsdropped.csv", encoding='utf8', sep=';')

In [3]:
df.head()

,Network,City,Country
0,10.100.131.0/27,Mumbai,India
1,10.100.131.160/27,Amsterdam,Netherlands
2,10.100.132.0/22,Mumbai,India
3,10.100.136.0/23,Mumbai,India
4,10.100.138.0/23,Mumbai,India


In [4]:
import json
import jmespath

regions = open('CityDatabase/regions.json', encoding ="utf8")

data = json.load(regions, encoding="utf8") 

# path = jmespath.search('countries[*].code',data)
# print(path)



In [5]:
def search_countrycode(x):
    query = """countries[?name==\'""" +  str(x).strip() + """\'].code"""
    path = jmespath.search(query,data)
    
    if(path==[]):
        return ''
    else:
        return path[0]

    
start = time.time()    
df['CountryCode'] = df['Country'].apply(search_countrycode)
end = time.time()

print(end - start)

20.66938090324402


In [6]:
df.head()

,Network,City,Country,CountryCode
0,10.100.131.0/27,Mumbai,India,IN
1,10.100.131.160/27,Amsterdam,Netherlands,NL
2,10.100.132.0/22,Mumbai,India,IN
3,10.100.136.0/23,Mumbai,India,IN
4,10.100.138.0/23,Mumbai,India,IN


In [7]:
df['City'].replace('', np.nan, inplace=True)
df['Country'].replace('', np.nan, inplace=True)
df['CountryCode'].replace('', np.nan, inplace=True)

In [8]:
# Find number of missing City or country and write them to a csv file

print("Number of rows before dropping missing city or country: "+str(len(df)))

df_null_city_or_country = df[(df['City'].isna() | df['Country'].isna())]

df_null_city_or_country.to_csv("Output/dropped02-gnoc_networks-nullcityorcountry.csv", index=False)

print("Number of rows with either missing city or country: "+str(len(df_null_city_or_country)))
 
df.dropna(subset=['Country','City'], inplace=True)

print("Number of rows after dropping missing city or country: "+str(len(df)))


Number of rows before dropping missing city or country: 15340
Number of rows with either missing city or country: 45
Number of rows after dropping missing city or country: 15295


In [9]:
# Find number of missing countrycodes and write them to a csv file

print("Number of rows before dropping missing country codes: "+str(len(df)))


df_missing_countrycode = df[df['CountryCode'].isna()]

df_missing_countrycode.to_csv("Output/dropped03-gnoc_networks-countrynotfound.csv", index=False)


print("Number of rows with missing country code: "+str(len(df_missing_countrycode)))
 
df.dropna(subset=['CountryCode'], inplace=True)

print("Number of rows after dropping missing city or country: "+str(len(df)))

Number of rows before dropping missing country codes: 15295
Number of rows with missing country code: 534
Number of rows after dropping missing city or country: 14761


In [10]:
df.to_csv("Output/output03-gnoc_networks-countrycodes.csv", sep=';', index=False)

#logger.warning("CSV generated: Output/output03-gnoc_networks-countrycodes.csv ")